# GWAS integration: enrichment and colocalization

This workflow processes fine-mapping results for xQTL, generated by `susie_twas` in the `mnm_regression.ipynb` notebook for cis xQTL, and GWAS fine-mapping results produced by `susie_rss` in the `rss_analysis.ipynb` notebook. It is designed to perform enrichment and colocalization analysis, particularly when fine-mapping results originate from different regions in the case of cis-xQTL and GWAS. The pipeline is capable to integrate and analyze data across these distinct regions. Originally tailored for cis-xQTL and GWAS integration, this pipeline can be applied to other pairwise integrations. An example of such application is in trans analysis, where the fine-mapped regions might be identical between trans-xQTL and GWAS, representing a special case of this broader implementation.

## Input

Lists of SuSiE fine-mapping output objects, in RDS format, of `class(susie)` in R. 
- For xQTL the list is meta-data of format: `chr`, `start`, `end`, `original_data`, `conditions_top_loci`, `block_top_loci` where `original_data` is an RDS file, `conditions_top_loci` is showing which contexts have top loci table (potential signals) `block_top_loci` is the blocks have overlapped top loci variant with xQTL data. That file could be output from `fine_mapping_post_processing/overlap_qtl_gwas`.
- For GWAS the list is meta-data of format: `chr`, `start`, `end`, `original_data`, `block_top_loci...` where `original_data` is an RDS file. That file could be output from `fine_mapping_post_processing/gwas_results_export`.
- Context meta is a metafile that shows the cohorts and the contained contexts. It can be used to identify the corresponding raw data for each context. 

## Analytical Logic 
1. Identify contexts within the xQTL meta-data that contain top loci results for each gene (condition_top_loci). Retrieve their corresponding xQTL original files (original_data) by referencing back to the context meta data. **Also, determine the GWAS blocks containing overlapping top loci variants for each gene (block_top_loci).**
2. Use the GWAS list to locate the original file names (original_data) for the identified block candidates.
3. Execute `xqtl_gwas_enrichment` to combine all QTL and GWAS original susie rds files for each context, enabling enrichment analysis and generating priors for subsequent coloc analysis. **Or**
4. Utilize `susie_coloc` to conduct coloc analysis for the selected xQTL and GWAS original files, analyzing each gene under each condition. This procedure automatically initiates enrichment analysis.

## Output

1. Enrichment analysis results --- this is a global enrichment estimate that combines all input data for each context. 
2. Colocalization results for regions of interest --- if `ld_meta_file_path` is provided, would also report coloc cs by coloc postprocessing. 


## Example
enrichment
```
sos run ~/codes/xqtl-pipeline/pipeline/SuSiE_enloc.ipynb xqtl_gwas_enrichment   \
    --gwas_meta_data  /mnt/vast/hpc/csg/rf2872/Work/pecotmr/encoloc_test/gwas.block_results_db.tsv     \
    --xqtl_meta_data  /mnt/vast/hpc/csg/rf2872/Work/Multivariate/gwas/overlap_test/ROSMAP_eQTL.overlapped.gwas.tsv         \
    --xqtl_finemapping_obj preset_variants_result susie_result_trimmed              \
    --xqtl_varname_obj preset_variants_result variant_names             \
    --gwas_finemapping_obj AD_Bellenguez_2022 single_effect_regression susie_result_trimmed          \
    --gwas_varname_obj  AD_Bellenguez_2022 single_effect_regression variant_names         \
    --xqtl_region_obj  region_info   grange   \
    --qtl-path /mnt/vast/hpc/csg/rf2872/Work/Multivariate/susie_2024_new/rds_files         \
    --gwas_path /mnt/vast/hpc/csg/hs3393/RSS_QC/GWAS_finemapping_Feb22_7dataset/SuSiE_RSS \
    --context_meta /mnt/vast/hpc/homes/rf2872/codes/fungen-xqtl-analysis/resource/context_meta.tsv 
```

coloc (would trigger enrichment automatically)
```
 sos run ~/codes/xqtl-pipeline/pipeline/SuSiE_enloc.ipynb susie_coloc    \
    --gwas_meta_data  /mnt/vast/hpc/csg/rf2872/Work/pecotmr/encoloc_test/gwas.block_results_db.tsv         \
    --xqtl_meta_data  /mnt/vast/hpc/csg/rf2872/Work/Multivariate/gwas/overlap_test/ROSMAP_eQTL.overlapped.gwas.tsv      \
    --xqtl_finemapping_obj preset_variants_result susie_result_trimmed              \
    --xqtl_varname_obj preset_variants_result variant_names             \
    --gwas_finemapping_obj AD_Bellenguez_2022 single_effect_regression susie_result_trimmed              \
    --gwas_varname_obj  AD_Bellenguez_2022 single_effect_regression variant_names             \
    --xqtl_region_obj  region_info   grange       \
    --qtl-path /mnt/vast/hpc/csg/rf2872/Work/Multivariate/susie_2024_new/rds_files             \
    --gwas_path /mnt/vast/hpc/csg/hs3393/RSS_QC/GWAS_finemapping_Feb22_7dataset/SuSiE_RSS     \
    --context_meta /mnt/vast/hpc/homes/rf2872/codes/fungen-xqtl-analysis/resource/context_meta.tsv      \
    --ld_meta_file_path /mnt/vast/hpc/csg/data_public/20240120_ADSP_LD_matrix/ld_meta_file.tsv 
```


eg: `gwas_meta_data`  (output from `fine_mapping_post_processing/gwas_results_export`)

```
#chr	start	end	region_id	TSS	original_data	combined_data	combined_data_sumstats	conditions	conditions_top_loci
chr1	101384274	104443097	1_101384274-104443097	NA	1_101384274-104443097.susie_rss.rds	gwas.1_101384274-104443097.cis_results_db.export.rds	gwas.1_101384274-104443097.cis_results_db.export_sumstats.rds	NA	NA
chr19	44935906	46842901	19_44935906-46842901	NA	19_44935906-46842901.susie_rss.rds	gwas.19_44935906-46842901.cis_results_db.export.rds	gwas.19_44935906-46842901.cis_results_db.export_sumstats.rds	AD_Bellenguez_2022,AD_Jansen_2021,AD_Kunkle_Stage1_2019,AD_Wightman_Full_2021,AD_Wightman_Excluding23andMe_2021,AD_Wightman_ExcludingUKBand23andME_2021	AD_Wightman_ExcludingUKBand23andME_2021.qc_impute,AD_Wightman_ExcludingUKBand23andME_2021.qc_only	AD_Wightman_ExcludingUKBand23andME_2021.qc_impute
```


eg: `xqtl_meta_data` (output from `fine_mapping_post_processing/overlap_qtl_gwas`)


```
#chr	start	end	region_id	TSS	original_data	combined_data	combined_data_sumstats	conditions	conditions_top_loci	block_top_loci	final_combined_data
chr1	167600000	171480000	ENSG00000000457	169894266	MSBB_eQTL.ENSG00000000457.univariate_susie_twas_weights.rds, ROSMAP_Kellis_eQTL.ENSG00000000457.univariate_susie_twas_weights.rds	Fungen_xQTL.ENSG00000000457.cis_results_db.export.rds	Fungen_xQTL.ENSG00000000457.cis_results_db.export_sumstats.rds	BM_10_MSBB_eQTL,BM_22_MSBB_eQTL,BM_36_MSBB_eQTL,BM_44_MSBB_eQTL,Ast_Kellis_eQTL	BM_22_MSBB_eQTL,BM_44_MSBB_eQTL		
chr19	41840000	47960000	ENSG00000130203	44905790	ROSMAP_Kellis_eQTL.ENSG00000130203.univariate_susie_twas_weights.rds, ROSMAP_Bennett_Klein_pQTL.ENSG00000130203.univariate_susie_twas_weights.rds	Fungen_xQTL.ENSG00000130203.cis_results_db.export.rds	Fungen_xQTL.ENSG00000130203.cis_results_db.export_sumstats.rds	Mic_Kellis_eQTL,DLPFC_Bennett_pQTL	Mic_Kellis_eQTL	19_44935906-46842901	Fungen_xQTL.ENSG00000130203.cis_results_db.export.overlapped.gwas.rds
chr20	49934867	53560000	ENSG00000000419	50958554	MSBB_eQTL.ENSG00000000419.univariate_susie_twas_weights.rds	Fungen_xQTL.ENSG00000000419.cis_results_db.export.rds	Fungen_xQTL.ENSG00000000419.cis_results_db.export_sumstats.rds	BM_10_MSBB_eQTL,BM_22_MSBB_eQTL	BM_22_MSBB_eQTL	Fungen_xQTL.ENSG00000000419.cis_results_db.export.overlapped.gwas.rds
```
eg: `context_meta` (should be updated as more analysis is done)
```
cohort	context
KNIGHT_pQTL	Knight_pQTL_brain,Knight_eQTL_brain
MSBB_eQTL	BM_10_MSBB_eQTL,BM_22_MSBB_eQTL,BM_36_MSBB_eQTL,BM_44_MSBB_eQTL
MIGA_eQTL	MiGA_GTS_eQTL,MiGA_SVZ_eQTL,MiGA_THA_eQTL
```

In [ ]:
[global]
# Workdir
parameter: cwd = path("output")
# A list of file paths for fine-mapped GWAS results. 
parameter: gwas_meta_data = path
# A list of file paths for fine-mapped xQTL results. 
parameter: xqtl_meta_data = path
# Optional: if a region list is provide the enrichment analysis will be focused on provided region. 
# The LAST column of this list will contain the ID of regions to focus on
parameter: region_list = path()
# Optional: if a region name is provided 
# the analysis would be focused on the union of provides region list and region names
parameter: region_name = []
# It is required to input the name of the analysis
parameter: name = f"{xqtl_meta_data:bnn}.{gwas_meta_data:bnn}"
parameter: container = ""
import re
parameter: entrypoint= ('micromamba run -a "" -n' + ' ' + re.sub(r'(_apptainer:latest|_docker:latest|\.sif)$', '', container.split('/')[-1])) if container else ""
# For cluster jobs, number commands to run per job
parameter: job_size = 200
# Wall clock time expected
parameter: walltime = "5m"
# Memory expected: quite large for enrichment analysis but small for xQTL colocalization
parameter: mem = "16G"
# Number of threads
parameter: numThreads = 1
#Optional table name in xQTL RDS files to get fine mapping results (eg 'susie_result_trimmed ').
parameter: xqtl_finemapping_obj = []
#Optional table name in xQTL RDS files to get variable names (eg ' variant_names ').
parameter: xqtl_varname_obj = []
#Optional table name in GWAS RDS files to get fine mapping results (eg 'AD_Bellenguez_2022 single_effect_regression susie_result_trimmed ').
parameter: gwas_finemapping_obj = []
#Optional table name in GWAS RDS files to get variable names(eg 'AD_Bellenguez_2022 single_effect_regression variant_names ').
parameter: gwas_varname_obj = []
#Optional table name in xQTL RDS files to get region info (eg 'susie_result_trimmed region_info grange ').
parameter: xqtl_region_obj = []
#Optional table name in GWAS RDS files to get region info (eg 'AD_Bellenguez_2022 single_effect_regression region_info grange ').
parameter: gwas_region_obj = []
#Directory path for GWAS orignal finemapping results 
parameter: gwas_path = ''
#Directory path for xQTL orignal finemapping results 
parameter: qtl_path = ''
# a meta file showing the context and corresponding cohort
parameter: context_meta = path()

import os
import pandas as pd

def adapt_file_path(file_path, reference_file):
    """
    Adapt a single file path based on its existence and a reference file's path.

    Args:
    - file_path (str): The file path to adapt.
    - reference_file (str): File path to use as a reference for adaptation.

    Returns:
    - str: Adapted file path.

    Raises:
    - FileNotFoundError: If no valid file path is found.
    """
    reference_path = os.path.dirname(reference_file)

    # Check if the file exists
    if os.path.isfile(file_path):
        return file_path

    # Check file name without path
    file_name = os.path.basename(file_path)
    if os.path.isfile(file_name):
        return file_name

    # Check file name in reference file's directory
    file_in_ref_dir = os.path.join(reference_path, file_name)
    if os.path.isfile(file_in_ref_dir):
        return file_in_ref_dir

    # Check original file path prefixed with reference file's directory
    file_prefixed = os.path.join(reference_path, file_path)
    if os.path.isfile(file_prefixed):
        return file_prefixed

    # If all checks fail, raise an error
    raise FileNotFoundError(f"No valid path found for file: {file_path}")

def adapt_file_path_all(df, column_name, reference_file):
    return df[column_name].apply(lambda x: adapt_file_path(x, reference_file))

def group_by_region(lst, partition):
    # from itertools import accumulate
    # partition = [len(x) for x in partition]
    # Compute the cumulative sums once
    # cumsum_vector = list(accumulate(partition))
    # Use slicing based on the cumulative sums
    # return [lst[(cumsum_vector[i-1] if i > 0 else 0):cumsum_vector[i]] for i in range(len(partition))]
    return partition


In [ ]:
[get_analysis_regions: shared = "regional_data"]
import pandas as pd

def make_unique_data(row):
    """Make the GWAS data unique by removing duplicates."""
    unique_data = ','.join(set(row.split(',')))
    return unique_data

# def check_required_columns(df, required_columns):
#     """Check if the required columns are present in the dataframe."""
#     missing_columns = [col for col in required_columns if col not in df.columns]
#     if missing_columns:
#         raise ValueError(f"Missing required columns: {', '.join(missing_columns)}")

def map_blocks_to_data(blocks, mapping_dict):
    """Map blocks to data using a provided mapping dictionary."""
    mapped_data = ','.join(mapping_dict.get(block.strip(), 'NA') for block in blocks.split(','))
    return mapped_data

def process_dataframes(xqtl_meta_data, gwas_meta_data):
    """Process the XQTL and GWAS dataframes."""
    xqtl_df = pd.read_csv(xqtl_meta_data, sep="\t")
    # filter xQTL data with the ones have overlapped top loci variants with GWAS data
    xqtl_df = xqtl_df[xqtl_df['block_top_loci'].notna()]

    gwas_df = pd.read_csv(gwas_meta_data, sep="\t")
    gwas_df = gwas_df[gwas_df['region_id'].isin(xqtl_df['block_top_loci'].str.split(',').explode().unique())]

    region_to_combined_data = dict(zip(gwas_df['region_id'], gwas_df['original_data']))
    # and only consider the overlapped GWAS blocks
    xqtl_df['block_data'] = xqtl_df['block_top_loci'].apply(map_blocks_to_data, args=(region_to_combined_data,))

    return xqtl_df, gwas_df

def generate_meta_dataframe(meta_data_path):
    """Generate a new long metadata by converting to context:cohort (1:1)."""
    meta = pd.read_csv(meta_data_path, sep='\t')
    new_meta = pd.DataFrame()
    contexts = pd.unique(meta['context'].str.split(',', expand=True).stack().str.strip())

    for context in contexts:
        mask = meta['context'].str.contains(context)
        tmp = pd.DataFrame({
            'context': [context],
            'cohort': [','.join(meta.loc[mask, 'cohort'])]
        })
        new_meta = pd.concat([new_meta, tmp], ignore_index=True)

    return new_meta

def generate_condition_based_dataframe(xqtl_df):
    """Generate a new long table by converting to condition:orginal_data (1:1)."""
    # only consider the QTL contexts have top loci data
    conditions = pd.unique(xqtl_df['conditions_top_loci'].str.split(',', expand=True).stack().str.strip())
    new_df = pd.DataFrame()

    for condition in conditions:
        mask = xqtl_df['conditions_top_loci'].str.contains(condition)
        tmp = pd.DataFrame({
            'condition': [condition],
            'QTL_original_data': [','.join(xqtl_df.loc[mask, 'original_data'])],
            'GWAS_original_data': [','.join(xqtl_df.loc[mask, 'block_data'])]
        })
        new_df = pd.concat([new_df, tmp], ignore_index=True)

    return new_df

def merge_and_filter_dfs(new_df, new_meta):
    """Merge and filter the dataframes based on condition/context and cohort to pick the corresponding original files."""
    new_df = new_df.set_index(['condition', 'GWAS_original_data'])['QTL_original_data'].str.split(',', expand=True).stack().reset_index(name='QTL_original_data').drop('level_2', axis=1)
    new_df['cohort_prefix'] = new_df['QTL_original_data'].apply(lambda x: x.split('.')[0])

    merged_df = pd.merge(new_df, new_meta, left_on='condition', right_on='context')
    filtered_df = merged_df[merged_df['cohort_prefix'].str.strip() == merged_df['cohort'].str.strip()]

    filtered_df = filtered_df.drop(columns=['cohort_prefix', 'context']).drop_duplicates()
    filtered_df['GWAS_original_data'] = filtered_df['GWAS_original_data'].apply(make_unique_data)

    return filtered_df

def prepare_final_paths(filtered_df, qtl_path, gwas_path):
    """Prepare final paths for QTL and GWAS original data."""
    filtered_df['QTL_original_data'] = filtered_df['QTL_original_data'].apply(
        lambda x: ','.join(f"{qtl_path}/{file_name.strip()}" for file_name in x.split(','))
    )
    filtered_df['GWAS_original_data'] = filtered_df['GWAS_original_data'].apply(
        lambda x: ','.join(f"{gwas_path}/{file_name.strip()}" for file_name in x.split(','))
    )
    return filtered_df

xqtl_df, gwas_df = process_dataframes(xqtl_meta_data, gwas_meta_data)
new_meta = generate_meta_dataframe(context_meta)
new_df = generate_condition_based_dataframe(xqtl_df)
filtered_df = merge_and_filter_dfs(new_df, new_meta)
filtered_df = prepare_final_paths(filtered_df, qtl_path, gwas_path)
regional_data = {
    'data': [row['QTL_original_data'].split(',') for _, row in filtered_df.iterrows()],
    'conditions': [(f"{row['condition']}", *row['GWAS_original_data'].split(',')) for _, row in filtered_df.iterrows()]
}

In [ ]:
[xqtl_gwas_enrichment]
depends: sos_variable("regional_data")
stop_if(len(regional_data['data']) == 0, f'No files left for analysis')

meta = regional_data['conditions']
input: regional_data["data"], group_by = lambda x: group_by_region(x, regional_data["data"]), group_with = "meta"
output: f'{cwd:a}/{name}.{_meta[0]}.enrichment.txt'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output:bn}'
R: expand = '${ }', stdout = f"{_output:n}.stdout", stderr = f"{_output:n}.stderr", container = container, entrypoint = entrypoint
  # RDS files for GWAS data
  gwas_finemapped_data = c(${paths([x for x in _meta[1:len(_meta)]]):r,})
  # RDS files for xQTL data
  xqtl_finemapped_data = c(${paths([x for x in _input]):r,})
  ##FIXME: for now we are using orginal data as input, but one question is, since there are several contexts, and several orginal files, we don't kow which ones belongs to which
  ## if we use exported data, we need to add (preset) alpha and (preset) V in exported data, which would make it big again, and we still need to figure out above question in coloc step
  enrich_result = pecotmr::xqtl_enrichment_wrapper(gwas_files = gwas_finemapped_data, xqtl_files = xqtl_finemapped_data, 
                                              xqtl_finemapping_obj =  c("${_meta[0]}",${",".join(['"%s"' % x  for x in xqtl_finemapping_obj]) if len(xqtl_finemapping_obj) != 0 else "NULL"}), 
                                              xqtl_varname_obj =   c("${_meta[0]}",${",".join(['"%s"' % x  for x in xqtl_varname_obj]) if len(xqtl_varname_obj) != 0 else "NULL"}), 
                                              gwas_finemapping_obj =  c(${",".join(['"%s"' % x for x in gwas_finemapping_obj]) if len(gwas_finemapping_obj) != 0 else "NULL"}), 
                                              gwas_varname_obj =  c(${",".join(['"%s"' % x for x in gwas_varname_obj]) if len(gwas_varname_obj) != 0 else "NULL"}))
  writeLines(paste(names(enrich_result), unlist(enrich_result), sep = ":"), ${_output:ar})

In [ ]:
[susie_coloc]
depends: sos_variable("regional_data"), sos_step("xqtl_gwas_enrichment")
stop_if(len(regional_data['data']) == 0, f'No files left for analysis')

parameter: ld_meta_file_path = path()
meta = regional_data['conditions']
input: regional_data["data"], group_by = lambda x: group_by_region(x, regional_data["data"]), group_with = "meta"
# output: f'{cwd:a}/{step_name[:-2]}/{name}.{_meta[0]}.coloc.rds'
output: f'{cwd:a}/{step_name}/{name}.{_meta[0]}.coloc.rds'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output:bn}'
R: expand = '${ }', stdout = f"{_output:n}.stdout", stderr = f"{_output:n}.stderr", container = container, entrypoint = entrypoint
    library(tidyverse)
    library(pecotmr)
    library(coloc)
    pkgs <- list.files("/mnt/vast/hpc/homes/rf2872/codes/pecotmr/R", full.names = TRUE)
    for(i in pkgs){
        source(i)
    }   
    # RDS files for xQTL data
    xqtl_finemapped_datas = c(${paths([x for x in _input]):r,})
    coloc_res <- list()
    # are we doing something like this? that means results are by each context, and each one has 
    for(xqtl_finemapped_data in xqtl_finemapped_datas){
      qtl_dat <- readRDS(xqtl_finemapped_data)
      gene = names(qtl_dat)
      gene_region = pecotmr:::get_nested_element(qtl_dat[[1]],  c("${_meta[0]}","region_info", "grange"))
  
      # Step 1: find relevant GWAS regions that overlap each the xQTL region of interest
      gwas_finemapped_datas <- c(${ ",".join(set('"%s"' % path for sublist in [','.join(x[1:]) for x in regional_data['conditions']] for path in sublist.split(',')))}) 
 
      gwas_regions <- gwas_finemapped_datas %>% basename %>% gsub('.susie_rss.rds','',.)
      overlap_index <- NULL
      for (i in 1:length(gwas_regions)) {
         print(i)
        region <- gwas_regions[i]
        split_region <- unlist(strsplit(region, "_"))
        block_chrom <- as.numeric(split_region[1])
        block_start <- as.numeric(split_region[2] %>% strsplit(., "-") %>% unlist %>% .[1])
        block_end <- as.numeric(split_region[2] %>% strsplit(., "-") %>% unlist %>% .[2])
        if (gene_region$chrom == block_chrom && (gene_region$start <= block_end |  gene_region$end >= block_start)) {
          overlap_index <- c(overlap_index, i)
        }
      }
  
     if (!is.null(overlap_index)) {
       gwas_finemapped_data <- gwas_finemapped_datas[overlap_index]
  
       # Step 2: load enrichment analysis results
       # Function to extract the numeric value for a given parameter name
       get_coloc_prior <- function(param_name, lines) {
         line <- grep(paste0(param_name, ":"), lines, value = TRUE)
         numeric_part <- as.numeric(gsub(paste0(".*", param_name, ":"), "", line))
         return(numeric_part)
       }
        # Extract values for p1, p2, and p12
       enrich_file = paste0('${cwd:a}','/', '${name}', '.' ,'${_meta[0]}','.enrichment.txt')
       p1 <- get_coloc_prior("p1", readLines(enrich_file))
       p2 <- get_coloc_prior("p2", readLines(enrich_file))
       p12 <- get_coloc_prior("p12", readLines(enrich_file))

       message("Priors are P1:", p1, "; p2: ", p2, "; p12: ", p12)
       # Step 3: Apply colocalization analysis between each condition and GWAS

       coloc_res[[gene]] <- coloc_wrapper(xqtl_file = xqtl_finemapped_data, gwas_files = gwas_finemapped_data, 
                                          xqtl_finemapping_obj =  c("${_meta[0]}",${",".join(['"%s"' % x  for x in xqtl_finemapping_obj]) if len(xqtl_finemapping_obj) != 0 else "NULL"}), 
                                          xqtl_varname_obj =   c("${_meta[0]}",${",".join(['"%s"' % x  for x in xqtl_varname_obj]) if len(xqtl_varname_obj) != 0 else "NULL"}), 
                                          gwas_finemapping_obj =  c(${",".join(['"%s"' % x for x in gwas_finemapping_obj]) if len(gwas_finemapping_obj) != 0 else "NULL"}), 
                                          gwas_varname_obj =  c(${",".join(['"%s"' % x for x in gwas_varname_obj]) if len(gwas_varname_obj) != 0 else "NULL"}),
                                          xqtl_region_obj =   c("${_meta[0]}",${",".join(['"%s"' % x  for x in xqtl_region_obj]) if len(xqtl_region_obj) != 0 else "NULL"}), 
                                          gwas_region_obj =  c(${",".join(['"%s"' % x for x in gwas_region_obj]) if len(gwas_region_obj) != 0 else "NULL"}),
                                          p1 = p1, p2 = p2, p12 = p12)

        if (${"TRUE" if ld_meta_file_path.is_file() else "FALSE"}) {
          coloc_res[[gene]] <- coloc_post_processor(coloc_res[[gene]], LD_meta_file_path = ${ld_meta_file_path:r}, analysis_region = coloc_res[[gene]]$analysis_region)
        }
      
      } else {
        print("No overlap found")
        coloc_res <-  "No overlap found"
      }
    }
    saveRDS(coloc_res, ${_output:r})